In [3]:
import os
import getpass


In [47]:

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

---
# TOC

### 1. Save Prompts as files
### 2. Using LangSmith for Prompt Management

---

# 1. Serialisation of Prompts in LangChain

🌐 **Prompt Serialisation in LangChain:**

- 📁 **Save & Share Easily**: Turn prompts into files for simple storage and sharing.

- 📄 **Choose Your Format**: Use JSON or YAML for easy-to-read files.

- 🛠️ **Flexible Storage**: Keep all your data in one place or spread it out—it's up to you.

- ⚡ **One-Stop Loading**: Regardless of the prompt type, loading them is a breeze.

**Core Principles:**

1. 👀**Readable by Humans**: JSON and YAML make prompts easy for us to read and edit.

2. 📚 **Flexible Filing**: Whether you're a one-file wonder or a multiple-file maestro, LangChain's got you covered.

With LangChain, managing and exchanging prompts is as smooth as sending an email!


# Saving prompts

In [52]:
from langchain_core.prompts import PromptTemplate

template = """You are an insightful question answering bot. A user will submit\
questions, which are delimited by triple backticks, and you should respond in\
an insighful manner.

Question: ```{question}```

Think step by step before answering.

Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
prompt.save("../data/saved/cot_prompt.json")

In [53]:
!cat ../data/saved/cot_prompt.json

{
    "name": null,
    "input_variables": [
        "question"
    ],
    "optional_variables": [],
    "output_parser": null,
    "partial_variables": {},
    "metadata": null,
    "tags": null,
    "template": "You are an insightful question answering bot. A user will submitquestions, which are delimited by triple backticks, and you should respond inan insighful manner.\n\nQuestion: ```{question}```\n\nThink step by step before answering.\n\nAnswer:\n",
    "template_format": "f-string",
    "validate_template": false,
    "_type": "prompt"
}

# Loading prompts

In [54]:
# All prompts are loaded through the `load_prompt` function.
from langchain_core.prompts import load_prompt

In [55]:
prompt = load_prompt("../data/saved/cot_prompt.json")

In [ ]:
print(prompt.template) # type: ignore # this line is to avoid mypy error

You are an insightful question answering bot. A user will submitquestions, which are delimited by triple backticks, and you should respond inan insighful manner.

Question: ```{question}```

Think step by step before answering.

Answer:



# 2. Using LangSmith for Prompt Management


### 🌐 **Pulling Version-Specific Prompts**

- LangChain hub allows fetching prompts by their exact commit hash.

- Guarantees use of the precise prompt version your app needs.

### 👩‍💻 **How to Specify Prompt Version**

- Add 'version' tag to the prompt ID in the pull command.

- Example in Python:
  - ```python
    from langchain import hub

    hub.pull(f"{handle}/{prompt-repo}:{version}")
    ```

### 🔑 **Getting Started Prerequisites**

- Must have a LangSmith account and an organization API key.

- Newbies can refer to [LangSmith documentation](https://docs.langchain.com/langsmith/home) for setup guidance.

Implement these steps for more dependable and controlled use of LangChain in production.

In [ ]:
os.environ["LANGCHAIN_ENDPOINT"] = "https://eu.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key:")

[Managing and accessing prompts in your code](https://docs.langchain.com/langsmith/manage-prompts-programmatically?_gl=1*iqjl1z*_ga*MTkzNzkxNzE4Ni4xNzYwMDg5NzY5*_ga_GEEHR1LQNV*czE3NjM0Nzc2MTAkbzExJGcxJHQxNzYzNDc4NjU2JGo1MSRsMCRoMA..)

In [43]:
from langsmith import Client

client = Client(api_url=os.environ["LANGCHAIN_ENDPOINT"])

In [ ]:
# Pushing a prompt to LangSmith 

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
url = client.push_prompt("joke-generator", object=prompt)
# url is a link to the prompt in the UI
print(url)

https://eu.smith.langchain.com/prompts/joke-generator/b9244726?organizationId=cc91df9b-ee35-4327-97aa-aa60f6e768cf


### Loading a Specific Version of a Prompt:

1️⃣ **Understanding Version Tracking:**

- Every update in a prompt repository is tagged with a unique commit hash.

2️⃣ **Default to Latest:**

- Accessing a repo automatically loads its most recent prompt version.

3️⃣ **Selecting a Specific Version:**

- Include the commit hash for a precise version when loading.

- Example: Use `c9839f14` to load a specific version of your prompt, eg. `prompt-ai-code-writer:c9839f14`.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-nano-2025-08-07")

prompt = client.pull_prompt("prompt-ai-code-writer")


# using LCEL
chain = prompt | llm

chain.invoke({"question": "How do I implement a transformer model using Hugging Face's Transformers library?"})

AIMessage(content='A clean way to implement and fine-tune a transformer with Hugging Face Transformers is to use the Auto classes, the datasets library for data, and the Trainer API for training. Here’s a compact guide plus a minimal runnable example.\n\nWhat you’ll do\n- Pick a task and a pre-trained model (e.g., text classification with DistilBERT).\n- Prepare data and convert labels to a “labels” column if using HF Trainer.\n- Tokenize inputs with the model’s tokenizer.\n- Define a model for your task (AutoModelForSequenceClassification, etc.).\n- Set up TrainingArguments and Trainer; provide a metric function.\n- Train, evaluate, and save the model.\n- (Optional) use a Pipeline for quick inference or push to Hugging Face Hub.\n\n Quick start (fine-tune a text classifier)\n- Install: pip install transformers datasets evaluating\n- Python script (use a small subset of data for a quick run)\n\nCode example (plain text, minimal runnable)\n- This fine-tunes DistilBERT on a binary text c

You can also pull a prompt as a RunnableSequence of a prompt and a model. Just specify include_model when pulling the prompt. If the stored prompt includes a model, it will be returned as a RunnableSequence. Make sure you have the proper environment variables set for the model you are using.

In [48]:
chain = client.pull_prompt("prompt-ai-code-writer", include_model=True)
chain.invoke({"question": "How do I implement a transformer model using Hugging Face's Transformers library?"})

/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/json/decoder.py:345: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


AIMessage(content='High-level steps, then minimal working examples (fine-tuning for classification with Trainer, and generation). Use the Auto* classes whenever possible.\n\n1) Install\npip install -U transformers datasets accelerate\n\n2) Typical workflow\n- Choose a model & tokenizer (AutoTokenizer, AutoModel...).\n- Prepare and tokenize your dataset (datasets library or custom DataLoader).\n- Create DataCollatorWithPadding or custom collate_fn.\n- Instantiate the model (optionally adjust config, num_labels).\n- Use Trainer + TrainingArguments for easy fine-tuning, or write a custom PyTorch loop.\n- Save/load and run inference with pipeline() or model+tokenizer.\n\nExample A — fine-tune DistilBERT for binary classification (IMDB) using Trainer\nfrom transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer\nfrom datasets import load_dataset\n\n# 1. dataset\nraw = load_dataset("imdb")\n\n# 2. tokenizer\ntokenizer = AutoT